In [57]:
# imports 
import numpy as np
import pandas as pd
import os
import re
import math

import pandas as pd
import numpy as np
from bokeh.palettes import magma, Set3
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.models import HoverTool,WheelZoomTool, PanTool, ResetTool, Legend
from bokeh.models.widgets import Panel, Tabs
import hvplot.pandas
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

# Load the merged dataset

In [6]:
# define the path under which the file is located
path = 'C:/Data_Science_for_Life_Sciences_MASTER/programming1/programming_1_influenza/data/'

# read the file
df = pd.read_csv(path + 'influenza_pop_dens_merged.csv', sep = '\t')
df

,Unnamed: 0,virus_type,district,season,case_number,year,area [ha],population,population_density,population_density_bw,pop_exp,pop_dens_exp,area_exp,area [km^2],area ex [km^2]
0,0,-nicht erhoben-,AD,2000/01,0.0,2000,135732.0,185929.0,137.0,294.0,170214.78,125.35,135793.33,1357.9333,1357.93
1,1,-nicht ermittelbar-,AD,2000/01,0.0,2000,135732.0,185929.0,137.0,294.0,170214.78,125.35,135793.33,1357.9333,1357.93
2,2,Influenza A Virus,AD,2000/01,20.0,2000,135732.0,185929.0,137.0,294.0,170214.78,125.35,135793.33,1357.9333,1357.93
3,3,Influenza A(H1N1) Virus (vorpandemisch),AD,2000/01,0.0,2000,135732.0,185929.0,137.0,294.0,170214.78,125.35,135793.33,1357.9333,1357.93
4,4,Influenza A(H3N2) Virus,AD,2000/01,0.0,2000,135732.0,185929.0,137.0,294.0,170214.78,125.35,135793.33,1357.9333,1357.93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11347,11347,Influenza A(H1N1)pdm09 Virus,U,2021/22,0.0,2021,NaN,NaN,NaN,NaN,129148.73,1088.12,NaN,NaN,118.69
11348,11348,Influenza A(H1N2) Virus,U,2021/22,0.0,2021,NaN,NaN,NaN,NaN,129148.73,1088.12,NaN,NaN,118.69
11349,11349,Influenza C Virus,U,2021/22,0.0,2021,NaN,NaN,NaN,NaN,129148.73,1088.12,NaN,NaN,118.69
11350,11350,Influenza A Virus (zoonotisch),U,2021/22,0.0,2021,NaN,NaN,NaN,NaN,129148.73,1088.12,NaN,NaN,118.69


In [14]:
# drop not required columns
df = df.drop(columns = ['Unnamed: 0', 'area [ha]', 'population', 'population_density', 'area [km^2]'])
df.head()

,virus_type,district,season,case_number,year,population_density_bw,pop_exp,pop_dens_exp,area_exp,area ex [km^2]
0,-nicht erhoben-,AD,2000/01,0.0,2000,294.0,170214.78,125.35,135793.33,1357.93
1,-nicht ermittelbar-,AD,2000/01,0.0,2000,294.0,170214.78,125.35,135793.33,1357.93
2,Influenza A Virus,AD,2000/01,20.0,2000,294.0,170214.78,125.35,135793.33,1357.93
3,Influenza A(H1N1) Virus (vorpandemisch),AD,2000/01,0.0,2000,294.0,170214.78,125.35,135793.33,1357.93
4,Influenza A(H3N2) Virus,AD,2000/01,0.0,2000,294.0,170214.78,125.35,135793.33,1357.93


In [ ]:
# rename the 

# Analyse the seasonal trends for whole Baden-Württemberg for available time period

Assumptions were, that there is an increase in cases from beginning of the recordings to now. Further an increase in virus variant diversity was stated to happen in 2012. It was assumed that the increase in virus diversity led to higher general case numbers (vaccines were then probably not too effective anymore, and fitting ones pers season harder to predict). A sudden decrease is expected to happen due to the covid measurements in season 2020/21.

In [17]:
i = df[['year', 'case_number']].groupby(['year']).sum()
i
p5 = figure(title = 'total case numbers baden - württemberg', x_axis_label = 'years', y_axis_label = 'total case numbers')

p5.line(i.index.values, i.case_number.values, line_width=2)

show(p5)

Indeed the global trend shows an increase from 2000 onwards, with sudden decrease in 2020. The Increase in case numbers seems to be stronger for the time period 2013 to 2019. However there is a local maximum found for the year 2009 (season 2009/10). For all trends seen it is interesting to see how the virus types contribute before having a look on literature to search for possible reasons.

In [25]:
virus_types =df[['year', 'case_number', 'virus_type']].groupby(['year', 'virus_type']).sum()
virus_types = virus_types.reset_index()
virus_types.virus_type.nunique()
line_col_set3 = Set3[12]

p6 = figure(plot_width=1000,title = 'total case numbers per virus type in baden - württemberg', x_axis_label = 'years',

             y_axis_label = 'total case numbers',x_range=[2000,2022],

             tools="hover", tooltips="@year:  @case_number for @virus_type")

 

for (name, group), color in zip(virus_types.groupby('virus_type'), line_col_set3):

    cds = ColumnDataSource(data = group)

    p6.line(x = 'year', y = 'case_number',source=cds, line_color = color, line_width = 2, legend_label = name)

legend = p6.legend[0]

p6.center = [item for item in p6.center if not isinstance(item, Legend)]

p6.add_layout(legend, 'left')

show(p6)

The added dimension by splitting the case numbers into case number per virus types yields the following insights:
- for influenza A there indeed seems to be the prognosed trend of increasing case numbers over time. Influenza A contributes the most to the total case numbers.
- similar trends to influenza A are observed for Influenza B, but with earlier decrease in case numbers than 2020
- There is a category mixing Influenza A and B if it was not differenciated between the two, which shows similar trends as influenza B
- The high case numbers in year season 2009/10 were mainly caused by one virus type which did not cause high case numbers in the upcoming years. (Possible conclusions: The later vaccines covered this specific variant, the variant was negative selected, the strain was circulating in some regions only, and wasnt spread further)
- There are some other categories, which are either influenza C or not further differenciated cases

The categories not showing clear virus strain are not really informative, when observing the evolvments of virus types. For that reason The categories 'others', 'not recorded', 'not ascertainable' will be removed.

In [33]:
vt_out = ['-nicht erhoben-', '-nicht ermittelbar-', '-andere/sonstige-']

df_filtered_vt = df.query('@vt_out not in virus_type')

# Study the virus spreading on district level

Study each virus variant per district. The difficulty here is, to see trends for 43 districts (normally 44 but RT was exlcuded), therefore a categorization was conducted. The categorization was conducted to support answering the question, whether the spreading patterns differentiate between rural and urban areas.

First plan was to separate the districts by SK and LK. SK means city district, LK means rural district. However when reading in the data it was intuitivly seen that some SK's do not have a much higher population density than the the LK's. Therfore, the decision was made to conduct the categorization based on the popualtion density.

Literature distinguishes between rural, suburban and urban areas. Based on literature urban areas are characterized by population density above 1500 and population above 50 000. Suburban population density above 300 and population above 500. Rural areas: population density below 300 and population below 500. This approach is applied to rasters, and it was not really sure whether it is applicable for this case as well. [https://ec.europa.eu/regional_policy/sources/docgener/work/2014_01_new_urban.pdf]

Therefore, it was chosen to follow an investigative approach to find the categories.

In [52]:
p7 = df.hvplot.violin(y = 'pop_dens_exp', ylabel = 'population density [i/km^2]', 
                                            title = 'distribution of population density in Baden-Württemberg')

p7

:Violin   (pop_dens_exp)

The violin plot shows that the majority of districts has a popualtion below 500 inhabitants per square kilometer. Several districts have a population density ranging between 500 inhabitants per square kilometer and about 2250 inhabitants per quare kilometer. A quite separated group ranges between 2500 inhabitants per square kilometers and 3120 square kilometers.

Instead of separating in the two groups urban and rural, a separation in 5 parts will be conducted.

From the previously mentioned literature, 0 - 300: rural, 300 - 1500: suburban, > 1500: urban. Since there are a lot of rural areas, i will introduce more categories, to distinguish there a bit more: sparse: 0 - 150, rural: 100 - 300, subrural: 300 - 500, suburban: 500 - 1500, urban: 1500 - inf.

The categories will be build base upon the population density and added as the column 'dem_dim' (demographic dimension) to the merged dataset.

In [47]:
# add the category column to the dataset
df['dem_dim'] = pd.cut(df['pop_dens_exp'], bins=[0, 150, 300, 500, 1500, float('Inf')], 
                                       labels=['sparse', 'rural', 'subrural', 'suburban', 'urban'])

In [50]:
t = np.where(df.groupby('district')['dem_dim'].nunique() > 1)
df[df.dem_dim == 'subrural'].groupby('district')['dem_dim'].nunique()

district
BAD    1
BS     1
ENZ    1
GP     1
HNL    1
KAL    1
KN     1
RA     1
RM     1
RN     1
TUE    1
Name: dem_dim, dtype: int64

In [51]:
total_case_distr = df.groupby(['district', 'year']).sum().reset_index()
total_case_distr

,district,year,case_number,population_density_bw,pop_exp,pop_dens_exp,area_exp,area ex [km^2]
0,AD,2000,20.0,3528.0,2042577.36,1504.20,1629519.96,16295.16
1,AD,2001,14.0,3564.0,2059595.52,1516.68,1629519.96,16295.16
2,AD,2002,34.0,3576.0,2076613.80,1529.28,1629519.96,16295.16
3,AD,2003,7.0,3588.0,2093631.96,1541.76,1629519.96,16295.16
4,AD,2004,18.0,3600.0,2110650.12,1554.36,1629519.96,16295.16
...,...,...,...,...,...,...,...,...
941,WT,2017,138.0,3708.0,2030035.68,1794.72,1357383.96,13573.80
942,WT,2018,178.0,3720.0,2042523.96,1805.76,1357383.96,13573.80
943,WT,2019,9.0,3732.0,2055012.24,1816.80,1357383.96,13573.80
944,WT,2020,0.0,3732.0,2067500.52,1827.84,1357383.96,13573.80


In [58]:
# choose line colors
p8_col = Set3[12]

# make tabbed line plot
for it, district_group in enumerate(df.dem_dim.unique()):
    data = df[df.dem_dim == district_group]
    p8 = figure(title = 'case number per district')
    dfs = data.groupby(['district', 'year']).sum().reset_index()
    for (name, group), color in zip(dfs.groupby(['district']), p8_col):
        p8.line(x = group.year, y = group.case_number, line_color = color, line_width = 2, legend_label = str(name))
    
    tab = Panel(child = p8, title = '{}'.format(district_group))
    if it == 0:
        tabs_list = [tab]
        tabs = Tabs(tabs=tabs_list)
        
    # append the other plots as tabs
    else:
        tabs_list.append(tab)
        tabs.update(tabs=tabs_list)

tabs = Tabs(tabs = tabs_list)
show(tabs)

There are no clear trends per district group seen. The plan was to conduct smoothing to compare whether there are significant differences between the categories. However this seems not to be meaningful when keeping the visualisations in mind. Maybe there can be trends found when selecting by virus variant.